In [10]:
## imports
import pandas as pd
import numpy as np
import plotnine
from plotnine import *
import random

## print multiple things from same cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from datetime import datetime, timedelta

## Load data

In [11]:
## load data on 2020 crimes in DC
df = dc_crim_2020 = pd.read_csv("https://opendata.arcgis.com/datasets/f516e0dd7b614b088ad781b0c4002331_2.csv")

## create report_dt column
df['report_dt'] = pd.to_datetime(df.REPORT_DAT)

## Warm-up Demo

In [3]:
%%time
for i in range(df.shape[0]):
    r = df.iloc[i]
    r.X + r.Y

CPU times: user 3.56 s, sys: 9.01 ms, total: 3.56 s
Wall time: 7.14 s


In [4]:
%%time
for i,r in df.iterrows():
    r.X + r.Y

CPU times: user 1.41 s, sys: 11 ms, total: 1.42 s
Wall time: 2.87 s


In [5]:
%%time
df.apply(lambda r: r.X + r.Y, axis = 1)

CPU times: user 532 ms, sys: 19 ms, total: 551 ms
Wall time: 1.12 s


0       -38.048603
1       -38.115721
2       -38.097421
3       -38.050871
4       -38.051294
           ...    
27927   -38.139043
27928   -38.055147
27929   -38.088756
27930   -38.110882
27931   -38.106803
Length: 27932, dtype: float64

In [6]:
%%time
## Super fast, but only works with built-in numpy functions.
df.X + df.Y

CPU times: user 1.26 ms, sys: 14 µs, total: 1.27 ms
Wall time: 1.19 ms


0       -38.048603
1       -38.115721
2       -38.097421
3       -38.050871
4       -38.051294
           ...    
27927   -38.139043
27928   -38.055147
27929   -38.088756
27930   -38.110882
27931   -38.106803
Length: 27932, dtype: float64

# Practice

In [12]:
## define crimes to look for and crimes to look within
## CCN is Central Complaint Number: https://go.mpdconline.com/GO/GO_401_01.pdf
CCN_examples = ['20165648', '20123250']
C_Tar = C_Target = crimes_lookfor = df[df.CCN.astype(str).isin(CCN_examples)][['CCN', 'WARD', 'OFFENSE', 'report_dt']]
C_Oth = C_Other  = other_crimes = df[~df.CCN.astype(str).isin(CCN_examples)]

## print crimes_lookfor
C_Tar.head()
# other_crimes.head()

,CCN,WARD,OFFENSE,report_dt
12639,20123250,2,MOTOR VEHICLE THEFT,2020-08-29 05:00:25+00:00
19204,20165648,6,MOTOR VEHICLE THEFT,2020-11-20 02:25:50+00:00


**Task**: we have two crimes we want to look for. We want to look in the remaining crime reports for crime reports that are:

- Located in the same ward as the two focal crimes
- Reported at the same time as the focal crime or up to 1000 minutes later (changed from slides which stated 20 mins since crime ids changed since last time so this long bandwidth helps us find matches!)

Solutions compare two ways to solve:

- Using a for loop
- Using a function

## 1. Loop approach

In [8]:
## create empty container to store results 
store_matches = {}

## loop through two example crimes
for i in range(C_Tar.shape[0]): # same as 
    
    ## extract row
    r = one_row = C_Tar.iloc[i]

    ## first, subset to crimes in same ward
    same_wards = C_Oth[C_Oth.WARD == r.WARD]
    
    ## second, with those same-ward crimes, construct indicator for reported within 20 minutes
    ## (interpreting as after but could do either)
    ### substep: get time cutoff
    CUTOFF = r.report_dt +  timedelta(minutes=1200)
    
    ### substep: use that to subset
    same_wards_sametime = same_wards[(same_wards.report_dt >= r.report_dt) & 
                                    (same_wards.report_dt <= CUTOFF)].copy()
    
    ## third, store the results
    store_matches[str(one_row.CCN)] = same_wards_sametime
    
## finally, concatenate results into one df
all_matches = pd.concat(store_matches)
all_matches.head()

X          Y       CCN              REPORT_DAT  \
20123250 6048  -77.021929  38.899129  20123419  2020/08/29 17:15:19+00   
         11308 -77.050528  38.913354  20123422  2020/08/29 16:45:57+00   
         13946 -77.038491  38.913727  20401318  2020/08/29 14:29:59+00   
         16387 -77.040091  38.909646  20123389  2020/08/29 16:05:18+00   
         23620 -77.039824  38.905656  20123507  2020/08/29 22:04:46+00   

                  SHIFT  METHOD              OFFENSE  \
20123250 6048       DAY  OTHERS          THEFT/OTHER   
         11308      DAY  OTHERS         THEFT F/AUTO   
         13946      DAY  OTHERS          THEFT/OTHER   
         16387      DAY  OTHERS         THEFT F/AUTO   
         23620  EVENING  OTHERS  MOTOR VEHICLE THEFT   

                                                BLOCK    XBLOCK    YBLOCK  \
20123250 6048        700 - 799 BLOCK OF 7TH STREET NW  398098.0  136808.0   
         11308  2200 - 2399 BLOCK OF DECATUR PLACE NW  395618.0  138388.0   
         13946    1724 - 1799 BLOCK OF 17TH STREET NW  396662.0  138429.0   
         16387       1700 - 1799 BLOCK OF P STREET NW  396523.0  137976.0   
         23620       1700 - 1779 BLOCK OF M STREET NW  396546.0  137533.0   

                ...  CENSUS_TRACT VOTING_PRECINCT   LATITUDE  LONGITUDE  \
20123250 6048   ...        5801.0    Precinct 129  38.899121 -77.021926   
         11308  ...        4100.0     Precinct 13  38.913346 -77.050526   
         13946  ...        5302.0     Precinct 15  38.913720 -77.038489   
         16387  ...        5303.0     Precinct 15  38.909638 -77.040089   
         23620  ...       10700.0     Precinct 17  38.905648 -77.039822   

                            BID              START_DATE  \
20123250 6048          DOWNTOWN  2020/08/29 16:05:40+00   
         11308              NaN  2020/08/26 22:00:29+00   
         13946              NaN  2020/08/28 20:55:00+00   
         16387              NaN  2020/08/28 22:00:23+00   
         23620  GOLDEN TRIANGLE  2020/08/27 19:01:24+00   

                              END_DATE   OBJECTID  OCTO_RECORD_ID  \
20123250 6048   2020/08/29 16:08:33+00  500137941             NaN   
         11308  2020/08/27 12:00:51+00  500147812             NaN   
         13946  2020/08/28 21:05:00+00  500157540             NaN   
         16387  2020/08/29 08:00:27+00  500183867             NaN   
         23620  2020/08/29 19:00:05+00  500366738             NaN   

                               report_dt  
20123250 6048  2020-08-29 17:15:19+00:00  
         11308 2020-08-29 16:45:57+00:00  
         13946 2020-08-29 14:29:59+00:00  
         16387 2020-08-29 16:05:18+00:00  
         23620 2020-08-29 22:04:46+00:00  

[5 rows x 26 columns]

# 1.5 Iterrow Approach

In [8]:
## create empty container to store results 
store_matches = {}

## loop through two example crimes
for i, r in C_Tar.iterrows(): # same as 

    ## subset to crimes in same ward
    same_wards = C_Oth[C_Oth.WARD == r.WARD]
    
    ## second, with those same-ward crimes, construct indicator for reported within 20 minutes
    ## (interpreting as after but could do either)
    ### substep: get time cutoff
    CUTOFF = r.report_dt +  timedelta(minutes=1200)
    
    ### substep: use that to subset
    same_wards_sametime = same_wards[(same_wards.report_dt >= r.report_dt) & 
                                    (same_wards.report_dt <= CUTOFF)].copy()
    
    ## third, store the results
    store_matches[str(one_row.CCN)] = same_wards_sametime
    
## finally, concatenate results into one df
all_matches = pd.concat(store_matches)
all_matches.head()

X          Y       CCN              REPORT_DAT  \
20165648 491   -77.015554  38.899950  20166039  2020/11/20 22:07:10+00   
         10824 -76.997328  38.885141  20165798  2020/11/20 12:46:32+00   
         14887 -77.005894  38.905167  20165986  2020/11/20 22:17:27+00   
         16277 -76.999518  38.891483  20165709  2020/11/20 04:27:36+00   
         16280 -77.001316  38.898915  20165932  2020/11/20 18:56:18+00   

                   SHIFT  METHOD              OFFENSE  \
20165648 491     EVENING  OTHERS          THEFT/OTHER   
         10824       DAY  OTHERS          THEFT/OTHER   
         14887   EVENING  OTHERS  MOTOR VEHICLE THEFT   
         16277  MIDNIGHT  OTHERS  MOTOR VEHICLE THEFT   
         16280       DAY  OTHERS         THEFT F/AUTO   

                                                     BLOCK    XBLOCK  \
20165648 491    300 - 363 BLOCK OF MASSACHUSETTS AVENUE NW  398651.0   
         10824   600 - 669 BLOCK OF PENNSYLVANIA AVENUE SE  400232.0   
         14887          1151 - 1199 BLOCK OF 1ST STREET NE  399489.0   
         16277            100 - 199 BLOCK OF 5TH STREET NE  400042.0   
         16280              300 - 399 BLOCK OF G STREET NE  399886.0   

                  YBLOCK  ...  CENSUS_TRACT VOTING_PRECINCT   LATITUDE  \
20165648 491    136899.0  ...        5900.0    Precinct 143  38.899942   
         10824  135255.0  ...        6500.0     Precinct 89  38.885133   
         14887  137478.0  ...       10603.0    Precinct 144  38.905159   
         16277  135959.0  ...        8200.0     Precinct 89  38.891475   
         16280  136784.0  ...        8301.0     Precinct 83  38.898907   

                LONGITUDE           BID              START_DATE  \
20165648 491   -77.015552      DOWNTOWN  2020/11/20 17:30:16+00   
         10824 -76.997326  CAPITOL HILL  2020/11/19 23:43:15+00   
         14887 -77.005891          NOMA  2020/11/20 20:15:26+00   
         16277 -76.999516           NaN  2020/11/20 03:02:27+00   
         16280 -77.001314           NaN  2020/11/20 15:30:02+00   

                              END_DATE   OBJECTID  OCTO_RECORD_ID  \
20165648 491    2020/11/20 22:08:28+00  499865401             NaN   
         10824                     NaN  500146815             NaN   
         14887  2020/11/20 21:46:24+00  500163163             NaN   
         16277                     NaN  500182525             NaN   
         16280  2020/11/20 18:25:35+00  500182528             NaN   

                               report_dt  
20165648 491   2020-11-20 22:07:10+00:00  
         10824 2020-11-20 12:46:32+00:00  
         14887 2020-11-20 22:17:27+00:00  
         16277 2020-11-20 04:27:36+00:00  
         16280 2020-11-20 18:56:18+00:00  

[5 rows x 26 columns]

## 2. Function approach

Practice rewriting the above loop as a function

### 2.1 define the function

In [9]:
store_matches_2 = {}

def find_related_crimes(r): # imagine the function taking in one row as its sole variable
    # Your code here
    

SyntaxError: incomplete input (1753950322.py, line 4)

In [25]:
#class done 

## create empty container to store results
store_matches_2 = {}
def new_crimes(r):
    same_wards = C_Oth[C_Oth.WARD == r.WARD]
    CUTOFF = r.report_dt +  timedelta(minutes=1200)
    same_wards_sametime = same_wards[(same_wards.report_dt >= r.report_dt) &
                                    (same_wards.report_dt <= CUTOFF)].copy()

    store_matches_2[str(r.CCN)] = same_wards_sametime





In [ ]:
#class done 

In [23]:
def find_related_crimes(row, crimes_lookfor, other_crimes):
    # Initialize an empty list to store matching crimes
    matching_crimes = []
    
    # Extract necessary information from the current row
    ward = row['WARD']
    report_dt = row['report_dt']
    
    # Filter other crimes based on the same ward
    ward_filtered = other_crimes[other_crimes['WARD'] == ward]
    
    # Filter other crimes based on time proximity (up to 1000 minutes)
    time_filtered = ward_filtered[
        (ward_filtered['report_dt'] >= report_dt) &
        (ward_filtered['report_dt'] <= report_dt + pd.Timedelta(minutes=1000))
    ]
    
    # Append the matching crimes to the list
    matching_crimes.extend(time_filtered['CCN'].tolist())
    
    return matching_crimes

# Apply the function to each row in crimes_lookfor and store the results
store_matches_2 = {}

for idx, row in crimes_lookfor.iterrows():
    store_matches_2[idx] = find_related_crimes(row, crimes_lookfor, other_crimes)

print(store_matches_2)


{12639: [20123419, 20123422, 20401318, 20123389], 19204: [20165798, 20165709, 20165932, 20165805, 20165859, 20165803]}


### 2.2 apply it to one of the focal crimes

In [27]:
def find_related_crimes(row, crimes_lookfor, other_crimes):
    # Create an empty DataFrame to store results
    store_matches = {}

    # Loop through each row in crimes_lookfor
    for i, r in crimes_lookfor.iterrows():
        # Subset to crimes in the same ward
        same_wards = other_crimes[other_crimes['WARD'] == r['WARD']]
        
        # Calculate the time cutoff
        CUTOFF = r['report_dt'] + timedelta(minutes=1200)
        
        # Subset to crimes reported within 20 minutes
        same_wards_sametime = same_wards[
            (same_wards['report_dt'] >= r['report_dt']) &
            (same_wards['report_dt'] <= CUTOFF)
        ].copy()
        
        # Store the results
        store_matches[str(r['CCN'])] = same_wards_sametime
    
    # Concatenate the results into one DataFrame
    all_matches = pd.concat(store_matches)
    
    return all_matches

# Apply the function to one of the focal crimes
result = find_related_crimes(C_Tar.iloc[0], C_Tar, C_Oth)
print(result.head())


                        X          Y       CCN              REPORT_DAT  \
20123250 6048  -77.021929  38.899129  20123419  2020/08/29 17:15:19+00   
         11308 -77.050528  38.913354  20123422  2020/08/29 16:45:57+00   
         13946 -77.038491  38.913727  20401318  2020/08/29 14:29:59+00   
         16387 -77.040091  38.909646  20123389  2020/08/29 16:05:18+00   
         23620 -77.039824  38.905656  20123507  2020/08/29 22:04:46+00   

                  SHIFT  METHOD              OFFENSE  \
20123250 6048       DAY  OTHERS          THEFT/OTHER   
         11308      DAY  OTHERS         THEFT F/AUTO   
         13946      DAY  OTHERS          THEFT/OTHER   
         16387      DAY  OTHERS         THEFT F/AUTO   
         23620  EVENING  OTHERS  MOTOR VEHICLE THEFT   

                                                BLOCK    XBLOCK    YBLOCK  \
20123250 6048        700 - 799 BLOCK OF 7TH STREET NW  398098.0  136808.0   
         11308  2200 - 2399 BLOCK OF DECATUR PLACE NW  395618.0 

In [28]:
# Select one of the focal crimes to apply the function
focal_crime = C_Tar.iloc[0]  # Selecting the first focal crime

# Apply the function to the selected focal crime
result = find_related_crimes(focal_crime, C_Tar, C_Oth)

# Display the results
print(result.head())

# this is correct 

                        X          Y       CCN              REPORT_DAT  \
20123250 6048  -77.021929  38.899129  20123419  2020/08/29 17:15:19+00   
         11308 -77.050528  38.913354  20123422  2020/08/29 16:45:57+00   
         13946 -77.038491  38.913727  20401318  2020/08/29 14:29:59+00   
         16387 -77.040091  38.909646  20123389  2020/08/29 16:05:18+00   
         23620 -77.039824  38.905656  20123507  2020/08/29 22:04:46+00   

                  SHIFT  METHOD              OFFENSE  \
20123250 6048       DAY  OTHERS          THEFT/OTHER   
         11308      DAY  OTHERS         THEFT F/AUTO   
         13946      DAY  OTHERS          THEFT/OTHER   
         16387      DAY  OTHERS         THEFT F/AUTO   
         23620  EVENING  OTHERS  MOTOR VEHICLE THEFT   

                                                BLOCK    XBLOCK    YBLOCK  \
20123250 6048        700 - 799 BLOCK OF 7TH STREET NW  398098.0  136808.0   
         11308  2200 - 2399 BLOCK OF DECATUR PLACE NW  395618.0 

In [ ]:
#class done 

### 2.3 Use apply to cover all the other focal crimes

In [21]:
results_all_focal = C_Tar.apply(lambda row: find_related_crimes(row, C_Tar, C_Oth), axis=1

print(results_all_focal.iloc[0].head())
                                
#wrong 

                        X          Y       CCN              REPORT_DAT  \
20123250 6048  -77.021929  38.899129  20123419  2020/08/29 17:15:19+00   
         11308 -77.050528  38.913354  20123422  2020/08/29 16:45:57+00   
         13946 -77.038491  38.913727  20401318  2020/08/29 14:29:59+00   
         16387 -77.040091  38.909646  20123389  2020/08/29 16:05:18+00   
         23620 -77.039824  38.905656  20123507  2020/08/29 22:04:46+00   

                  SHIFT  METHOD              OFFENSE  \
20123250 6048       DAY  OTHERS          THEFT/OTHER   
         11308      DAY  OTHERS         THEFT F/AUTO   
         13946      DAY  OTHERS          THEFT/OTHER   
         16387      DAY  OTHERS         THEFT F/AUTO   
         23620  EVENING  OTHERS  MOTOR VEHICLE THEFT   

                                                BLOCK    XBLOCK    YBLOCK  \
20123250 6048        700 - 799 BLOCK OF 7TH STREET NW  398098.0  136808.0   
         11308  2200 - 2399 BLOCK OF DECATUR PLACE NW  395618.0 

In [29]:
#class done 


C_Tar["WARD"]

12639    2
19204    6
Name: WARD, dtype: int64